In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.append('../')

In [60]:
import nets.cornet.cornet_s as CS
import nets.cornet.cornet_z as CZ

In [99]:
m = CZ.CORnet_Z()

In [89]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
m = Net()

In [50]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F

In [104]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                               transforms.Grayscale(3),
    transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

In [103]:
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.Compose([
                            transforms.Grayscale(3),
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                           
                       ]))

In [100]:
m = m.cuda()

In [112]:
LR = 1e-3
MOMENTUM = .9
WD=1e-4
optimizer = torch.optim.SGD(m.parameters(),
                            LR,
                            momentum=MOMENTUM,
                            weight_decay=WD)

torch.optim.lr_scheduler.StepLR(optimizer, step_size=10)
loss = torch.nn.CrossEntropyLoss().cuda()

In [106]:
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=128, shuffle=True)

In [110]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = torch.nn.CrossEntropyLoss().cuda()#F.nll_loss(output, target)
        loss = loss(output, target)
#         loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
#             print(data, target)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [113]:
args={'log_interval':10000}

for epoch in range(1, 10):
    
    train(args, m, 'cuda', train_loader, optimizer, epoch)
    test(args, m, 'cuda', test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 6.971705

Test set: Average loss: -24.6709, Accuracy: 9274/10000 (93%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.337479

Test set: Average loss: -23.3518, Accuracy: 9584/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.101002

Test set: Average loss: -23.5341, Accuracy: 9672/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.106650

Test set: Average loss: -23.6639, Accuracy: 9761/10000 (98%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.110379

Test set: Average loss: -23.8358, Accuracy: 9787/10000 (98%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.080337

Test set: Average loss: -23.6368, Accuracy: 9801/10000 (98%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.053814

Test set: Average loss: -23.8080, Accuracy: 9815/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.105850

Test set: Average loss: -23.9422, Accuracy: 9834/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.050605

Test set: Average loss: -24.1631, Accuracy: 9830/10000 (98%)



In [ ]:


target = target.cuda(non_blocking=True)
output = m(input)

record = {}
loss = self.loss(output, target)
record['loss'] = loss.item()
record['top1'], record['top5'] = accuracy(output, target, topk=(1, 5))
record['top1'] /= len(output)
record['top5'] /= len(output)
record['learning_rate'] = self.lr.get_lr()[0]

self.optimizer.zero_grad()
loss.backward()
self.optimizer.step()

record['dur'] = time.time() - start